In [114]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import textstat

In [115]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [116]:
df = pd.read_csv(r'C:\Users\jeanv\Documents\GitHub\jun24_bc_llm\df_random_summaries_llm_eval.csv')

df2 = pd.read_csv(r'C:\Users\jeanv\Documents\GitHub\jun24_bc_llm\df_preclassified_summaries_refFree_refBased.csv')

In [117]:
df['ratio']=df['ratio'].str.replace('%', '').astype(float)

filter_mask=(df['model_name']!="gpt3.5")&(df['ratio']<=150)&(df['duration']<=200)&(df['summary']!="Error timeout")

df=df[filter_mask]

filter_mask2=(df2['model_name']!="gpt3.5")&(df2['ratio']<=150)&(df2['duration']<=200)&(df2['summary']!="Error timeout")

df2=df2[filter_mask2]

df2 = df2[df2['error'].isnull()]

In [118]:
# Function to calculate evaluation metrics
def calculate_metrics(df):
    results = []
    
    for idx, row in df.iterrows():
        summary = row['summary']
        reference = row['ref_summary']
        
        # Calculate ROUGE scores
        rouge_scores = rouge_scorer.score(summary, reference)
        rouge1 = rouge_scores['rouge1'].fmeasure
        rouge2 = rouge_scores['rouge2'].fmeasure
        rougeL = rouge_scores['rougeL'].fmeasure
        
        # Calculate BLEU score
        reference_tokens = [reference.split()]
        summary_tokens = summary.split()
        smoothing = SmoothingFunction().method4
        bleu = sentence_bleu(reference_tokens, summary_tokens, smoothing_function=smoothing)
        
        # Calculate Flesch Reading Ease
        fre_score = textstat.flesch_reading_ease(summary)
        
        # Calculate Dale-Chall Readability
        dale_chall_score = textstat.dale_chall_readability_score(summary)
        
        # Store the results
        results.append({
            'rouge1': rouge1,
            'rouge2': rouge2,
            'rougeL': rougeL,
            'bleu': bleu,
            'flesch_reading_ease': fre_score,
            'dale_chall_readability': dale_chall_score
        })
    
    # Convert to a dataframe
    metrics_df = pd.DataFrame(results)
    return pd.concat([df, metrics_df], axis=1)

In [119]:
# Assuming your dataframe is 'df' with 'summary' and 'reference_summary' columns
df_with_metrics = calculate_metrics(df)

df2_with_metrics = calculate_metrics(df)